In [ ]:
# 라이브러리 모음
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.graph_objects as go
import plotly.express as px

# 모듈 모음
import modules as mds

In [ ]:
# Windows에서 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # '맑은 고딕'이 설치되어 있을 경우
plt.rcParams['axes.unicode_minus'] = False     # 마이너스(-) 부호 깨짐 방지

In [ ]:
# macOS에서 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'   # macOS 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False

# 설비용량 & 발전량 데이터

In [ ]:
# 누적 설비용량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total = pd.read_excel("dataset/지역별_누적_설비용량.xlsx")
data_2024 = pd.read_excel("dataset/2024년도_신규_설비용량.xlsx")
data_2023 = pd.read_excel("dataset/2023년도_신규_설비용량.xlsx")
data_2022 = pd.read_excel("dataset/2022년도_신규_설비용량.xlsx")
data_2021 = pd.read_excel("dataset/2021년도_신규_설비용량.xlsx")

# 데이터프레임으로 변환
# ic = Installed Capacity = 설비용량
ic_total = pd.DataFrame(data_total)
ic_2024 = pd.DataFrame(data_2024)
ic_2023 = pd.DataFrame(data_2023)
ic_2022 = pd.DataFrame(data_2022)
ic_2021 = pd.DataFrame(data_2021)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(ic_total)
mds.delete_comma(ic_2024)
mds.delete_comma(ic_2023)
mds.delete_comma(ic_2022)
mds.delete_comma(ic_2021)

# 컴럼 통일화를 위해 capacity_total에 "합계" 컬럼 추가
ic_total["합계"] = ic_total.sum(axis=1, numeric_only=True)

# 하나의 DataFrame으로 병합
ic_total = pd.concat([ic_total, ic_2024, ic_2023, ic_2022, ic_2021], ignore_index=True)

# 각 연도별 누적값 계산
for i in range(1, len(ic_total)):
    ic_total.iloc[i, 1:] = ic_total.iloc[i-1, 1:] - ic_total.iloc[i, 1:]

# 연도 정리
ic_total.rename(columns={"Unnamed: 0": "연도"}, inplace=True)
ic_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 2020-2024 각 연도의 지역별 누적 설비용량
ic_total

In [ ]:
# 누적 발전량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total_2022 = pd.read_excel("dataset/지역별_누적_발전량.xlsx")
data_2024 = pd.read_excel("dataset/2024년_신규_발전량.xlsx")
data_2023 = pd.read_excel("dataset/2023년_신규_발전량.xlsx")
data_2022 = pd.read_excel("dataset/2022년_신규_발전량.xlsx")
data_2021 = pd.read_excel("dataset/2021년_신규_발전량.xlsx")
data_2020 = pd.read_excel("dataset/2020년_신규_발전량.xlsx")

# 데이터프레임으로 변환
# gen = Power Generation = 발전량
gen_total_2022 = pd.DataFrame(data_total_2022)
gen_2024 = pd.DataFrame(data_2024)
gen_2023 = pd.DataFrame(data_2023)
gen_2022 = pd.DataFrame(data_2022)
gen_2021 = pd.DataFrame(data_2021)
gen_2020 = pd.DataFrame(data_2020)

# 열과 행 전치
gen_2024 = gen_2024.T.reset_index(drop=True)
gen_2023 = gen_2023.T.reset_index(drop=True)
gen_2022 = gen_2022.T.reset_index(drop=True)
gen_2021 = gen_2021.T.reset_index(drop=True)
gen_2020 = gen_2020.T.reset_index(drop=True)

# 컴럼 정리
gen_2024.columns = gen_2024.iloc[0]
gen_2024.drop(labels=gen_2024.index[0], axis=0, inplace=True)
gen_2023.columns = gen_2023.iloc[0]
gen_2023.drop(labels=gen_2023.index[0], axis=0, inplace=True)
gen_2022.columns = gen_2022.iloc[0]
gen_2022.drop(labels=gen_2022.index[0], axis=0, inplace=True)
gen_2021.columns = gen_2021.iloc[0]
gen_2021.drop(labels=gen_2021.index[0], axis=0, inplace=True)
gen_2020.columns = gen_2020.iloc[0]
gen_2020.drop(labels=gen_2020.index[0], axis=0, inplace=True)


# 연도 컬럼 추가
gen_2024.insert(0, "연도", np.nan)
gen_2023.insert(0, "연도", np.nan)
gen_2022.insert(0, "연도", np.nan)
gen_2021.insert(0, "연도", np.nan)
gen_2020.insert(0, "연도", np.nan)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(gen_total_2022)
mds.delete_comma(gen_2024)
mds.delete_comma(gen_2023)
mds.delete_comma(gen_2022)
mds.delete_comma(gen_2021)
mds.delete_comma(gen_2020)

# 월별 데이터 연도별 데이터로 변환
gen_2024 = pd.DataFrame(gen_2024.sum()).T
gen_2023 = pd.DataFrame(gen_2023.sum()).T
gen_2022 = pd.DataFrame(gen_2022.sum()).T
gen_2021 = pd.DataFrame(gen_2021.sum()).T
gen_2020 = pd.DataFrame(gen_2020.sum()).T

# "합계" 컬럼 추가
gen_2024["합계"] = gen_2024.sum(axis=1, numeric_only=True)
gen_2023["합계"] = gen_2023.sum(axis=1, numeric_only=True)
gen_2022["합계"] = gen_2022.sum(axis=1, numeric_only=True)
gen_2021["합계"] = gen_2021.sum(axis=1, numeric_only=True)
gen_2020["합계"] = gen_2020.sum(axis=1, numeric_only=True)

# 하나의 DataFrame으로 병합
gen_total = pd.concat([gen_2024, gen_2023, gen_total_2022, gen_2022, gen_2021], ignore_index=True)

# 연도 정리
gen_total = gen_total.drop(gen_total.columns[-1], axis=1)
gen_total.rename(columns={"지역": "연도"}, inplace=True)
gen_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 소수점 제거
gen_total = gen_total.astype(int)

# # 각 연도별 누적값 계산
gen_total.iloc[1, 1:] = gen_total.iloc[2, 1:] + gen_total.iloc[1, 1:] # 2023 계산
gen_total.iloc[0, 1:] = gen_total.iloc[1, 1:] + gen_total.iloc[0, 1:] # 2024 계산
gen_total.iloc[3, 1:] = gen_total.iloc[2, 1:] - gen_total.iloc[3, 1:] # 2021 계산
gen_total.iloc[4, 1:] = gen_total.iloc[3, 1:] - gen_total.iloc[4, 1:] # 2020 계산

# 2020-2024 각 연도의 지역별 누적 발전량
gen_total.head()

In [ ]:
# 연도별 신규 발전량 데이터 재구성

# 하나의 DataFrame으로 병합
gen_new_total = pd.concat([gen_2024, gen_2023, gen_2022, gen_2021, gen_2020], ignore_index=True).astype(int)
gen_new_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 2020-2024 연도별 신규 발전량
gen_new_total

In [ ]:
# 연도별 지역별 / 설비용량 발전량 이용률 데이터 정리

# 데이터 복사
capacity = ic_total.copy()
generation = gen_new_total.copy()

# 설비용량 long 변환 (합계 제외)
cap_long = capacity.drop(columns=["합계"]).melt(
    id_vars=["연도"], var_name="지역", value_name="설비용량(MW)"
)

# 발전량 long 변환 (합계 제외)
gen_long = generation.drop(columns=["합계"]).melt(
    id_vars=["연도"], var_name="지역", value_name="발전량(MWh)"
)

# 데이터 합치기
merged = pd.merge(cap_long, gen_long, on=["연도", "지역"])

# 설비 이용률 계산
merged["설비이용률(%)"] = merged["발전량(MWh)"] / (merged["설비용량(MW)"] * 8760) * 100

# 연도 기준 내림차순 (2024 → 2020)
merged_sorted = merged.sort_values(by="연도", ascending=False)

# 인덱스 초기화
merged_sorted = merged_sorted.reset_index(drop=True)

# 연도별 지역별 / 설비용량 발전량 이용률 데이터
print(merged_sorted.head(20))  # 상위 20개 확인

# 일사량 데이터

In [ ]:
# 24년 지역별 합계 일사량

solar_radiation_year = pd.read_csv('dataset/지역별_합계_일사량.csv', encoding="cp949")

# 컬럼명 정리
solar_radiation_year.rename(columns={"지점명": "지역"}, inplace=True)

# 지역명 정리
solar_radiation_year["지역"] = ["강원", "서울", "인천", "경기", "충북", "대전", "경북", "대구", "전북", "울산", "광주", "부산", "전남", "제주", "경남", "세종", "충남"]

# 필요한 컬럼으로 재구성
solar_radiation_year = solar_radiation_year.iloc[:, 1:]

# 지역 순서 통일
solar_radiation_year = solar_radiation_year.iloc[[1, 11, 7, 2, 10, 5, 9, 15, 3, 4, 16, 12, 6, 14, 13, 0, 8]].reset_index(drop=True).round()

# 일사량 값 정수로 변환
solar_radiation_year["합계 일사량(MJ/m2)"] = solar_radiation_year["합계 일사량(MJ/m2)"].astype(int)

# 2024 지역별 합계 일사량
solar_radiation_year

In [ ]:
# 2020-2024 지역별 힙계 일사량

solar_radiation_total_year = pd.read_csv('dataset/20-23_지역별_합계_일사량.csv', encoding="cp949")

# 컬럼명 정리
solar_radiation_total_year.rename(columns={"지점명": "지역"}, inplace=True)
solar_radiation_total_year = solar_radiation_total_year[["지역", "일시", "합계 일사량(MJ/m2)"]]

# 피벗으로 변환
solar_radiation_total_year = solar_radiation_total_year.pivot_table(index="일시", columns="지역", values="합계 일사량(MJ/m2)", aggfunc='max', fill_value=0).round().astype(int)

# 인덱스 컬럼 정리
solar_radiation_total_year.index.name = None
solar_radiation_total_year.columns.name = None
solar_radiation_total_year = solar_radiation_total_year.T

# 지역명 정리
solar_radiation_total_year["지역"] = ["강원", "광주", "대구", "대전", "부산", "서울", "세종", "경기", "경북", "전남", "울산", "인천", "전북", "제주", "경남", "충북", "충남"]
solar_radiation_total_year.reset_index(drop=True, inplace=True)
solar_radiation_total_year = solar_radiation_total_year[["지역", 2020, 2021, 2022, 2023]]

# 지역 순서 통일
solar_radiation_total_year = solar_radiation_total_year.iloc[[5,4,2,11,1,3,10,6,7,15,16,9,8,14,13,0,12]].reset_index(drop=True)

# 24년 데이터 추가
data = solar_radiation_year["합계 일사량(MJ/m2)"]
solar_radiation_total_year[2024] = data

# 평균 일사량 계산
solar_radiation_mean_year = solar_radiation_total_year.copy()
solar_radiation_mean_year.iloc[:,1:] = (solar_radiation_mean_year.iloc[:,1:] / 12).round().astype(int)

# 2020-2024 지역별 합계 일사량
solar_radiation_total_year

# 2020-2024 지역별 평균 일사량
solar_radiation_mean_year

# 합계 일사량 엑셀 파일 저장
# solar_radiation_total_year.to_excel("dataset/2020-2024_지역별_합계_일사량.xlsx", index=False, engine="openpyxl")

# 평균 일사량 엑셀 파일 저장
# solar_radiation_mean_year.to_excel("dataset/2020-2024_지역별_평균_일사량.xlsx", index=False, engine="openpyxl")

In [ ]:
# 충남 경남 경북 10년치 일사량

area_solar = pd.read_csv("dataset/15-19 충남 경남 경북 합계 일사량.csv", encoding="cp949")

# 20-24 합계 일사량 데이터 충남 경뷱 경남만 복사
area_solar_total_year = solar_radiation_total_year.iloc[[10,12,13],:].reset_index(drop=True)

# 컬럼명 정리 후 사용할 컬럼만 선택
area_solar.rename(columns={"지점명": "지역"}, inplace=True)
area_solar = area_solar[["지역", "일시", "합계 일사량(MJ/m2)"]]

# # 피벗으로 변환
area_solar = area_solar.pivot_table(index="일시", columns="지역", values="합계 일사량(MJ/m2)", aggfunc='max', fill_value=0).round().astype(int)

# # 인덱스 컬럼 정리
area_solar.index.name = None
area_solar.columns.name = None
area_solar = area_solar.T

# 지역 컬럼 추가
area_solar["지역"] = ["충남", "경북", "경남"]
area_solar.reset_index(drop=True, inplace=True)

# 컬럼 순서 정리
area_solar = area_solar[["지역", 2015, 2016, 2017, 2018, 2019]]

# df 결합
area_solar_total_year = pd.concat([area_solar, area_solar_total_year.iloc[:,1:]], axis=1)

# 평균값 구하기
area_solar_total_year.iloc[:,1:] = (area_solar_total_year.iloc[:,1:] / 12).round().astype(int)

# 2015-2024 충남 / 경북 / 경남 연도별 평균 일사량
area_solar_total_year

In [ ]:
# 24년 지역 / 월별 일사량

solar_radiation_month = pd.read_csv('dataset/지역별_월별_합계_일사량.csv', encoding="cp949")

# 피벗으로 변환
solar_radiation_month = solar_radiation_month.iloc[:, 1:]
solar_radiation_month = solar_radiation_month.pivot_table(index="일시", columns="지점명", values="합계 일사량(MJ/m2)", aggfunc='max', fill_value=0).round().astype(int)

# 인덱스 / 컬럼 정리
solar_radiation_month_df = solar_radiation_month.T.reset_index()
solar_radiation_month_df.index.name = None
solar_radiation_month_df.columns.name = None
solar_radiation_month_df.rename(columns={"지점명": "지역"}, inplace=True)

# 지역명 / 순서 통일
solar_radiation_month_df = solar_radiation_month_df.iloc[[6,5,2,12,1,3,11,7,0,16,4,10,9,15,14,0,13]].reset_index(drop=True)
solar_radiation_month_df["지역"] = ["서울","부산","대구","인천","광주","대전","울산","세종","경기","충북","충남","전남","경북","경남","제주","강원","전북"]

# 결측값 처리
avg = solar_radiation_month_df["2024-01"].mean().round()
avg = avg.astype(int)
solar_radiation_month_df.iloc[10, 1:3] = avg

# 2024 지역별 월별 평균 일사량
solar_radiation_month_df

# 발전소 데이터

In [ ]:
# 경남 태양광 발전소 현황

gn_solar = pd.read_csv("dataset/경상남도_태양광 발전설치현황.csv", encoding="cp949")

# 설비용량 높은 순서대로 정렬
gn_solar.sort_values(["설비용량(킬로와트)"],ascending=False, inplace=True)
gn_solar.reset_index(drop=True, inplace=True)

# 데이터 간소화
gn_solar = gn_solar.head(5)[["상호", "설비용량(킬로와트)", "설치장소"]]

# 컬럼명 통일
gn_solar.rename(columns={"상호": "발전소명", "설비용량(킬로와트)": "설비용량(KW)", "설치장소": "위치"}, inplace=True)

# 위도 / 경도 추가
gn_solar["위도"] = [128.9784927,128.1299150,128.7015250,128.0649578,129.0117573]
gn_solar["경도"] = [35.2816030,35.1150670,35.1921100,35.0774302,35.2925874]

# 경남 발전소 위치 및 용량
gn_solar

# 엑셀 파일 저장
# gn_solar.to_excel("dataset/경남_설비목록.xlsx", index=False, engine="openpyxl")

In [ ]:
# 경북 태양광 발전소 현황

gb_solar = pd.read_csv("dataset/경상북도_태양광발전소 현황.csv", encoding="cp949")

# 설비용량 높은 순서대로 정렬
gb_solar.sort_values(["사업개시용량"],ascending=False, inplace=True)
gb_solar.reset_index(drop=True, inplace=True)

# 데이터 간소화
gb_solar = gb_solar.head(5)[["발전소명", "사업개시용량", "설치장소"]]

# 컬럼명 통일
gb_solar.rename(columns={"발전소명": "발전소명", "사업개시용량": "설비용량(KW)", "설치장소": "위치"}, inplace=True)

# 위도 / 경도 추가
gb_solar["위도"] = [129.0736592,128.7316020,128.5980041,128.1383791,128.6374363]
gb_solar["경도"] = [35.9595092,35.7801541,36.2453381,36.7009351,36.8854929]

# 경북 발전소 위치 및 용량
gb_solar

# 엑셀 파일 저장
# gb_solar.to_excel("dataset/경북_설비목록.xlsx", index=False, engine="openpyxl")

In [ ]:
# 충남지역 발전소 현황

cn_solar = pd.read_csv("dataset/충청남도_발전사업 현황.csv", encoding="cp949")

# 설비용량 높은 순서대로 정렬
mask = cn_solar.iloc[:,-1] == "태양광"
cn_solar = cn_solar[mask].reset_index(drop=True)

# 데이터 간소화
cn_solar = cn_solar[[" 발전소명 ", "설치장소", " 허가용량(키로와트) "]]

# 데이터 정렬
cn_solar = cn_solar.sort_values([" 허가용량(키로와트) "], ascending=False).reset_index(drop=True)
cn_solar = cn_solar.iloc[[0,1,2,27,28],:].reset_index(drop=True)

# 위도 / 경도 추가
cn_solar["위도"] = [126.7547192,126.5105981,126.3309592,126.6998657,126.3935127]
cn_solar["경도"] = [36.9781090,37.0216299,36.6114134,36.7963187,36.8640524]

# 충남 발전소 위치 및 용량
cn_solar

# 엑셀 파일 저장
# cn_solar.to_excel("dataset/충남_설비목록.xlsx", index=False, engine="openpyxl")

# 통합 데이터

In [ ]:
# 최종 total 데이터 정리

# === 1) 데이터 불러오기 ===
df_main = merged_sorted.copy()
df_solar = solar_radiation_total_year.copy()

# === 2) 가로형 → 세로형 변환 ===
df_solar_long = df_solar.melt(
    id_vars="지역", 
    var_name="연도", 
    value_name="일사량(MJ/m2)"
)

# 연도 숫자형으로 변환
df_solar_long["연도"] = df_solar_long["연도"].astype(int)

# === 3) 두 데이터 합치기 ===
merged = pd.merge(
    df_main,
    df_solar_long,
    on=["연도", "지역"],
    how="left"
)

merged["일사량(MJ/m2)"] = merged["일사량(MJ/m2)"] / 12

# 결측값 처리
merged = merged[~merged['지역'].isin(['세종', '울산'])].reset_index(drop=True)

# 최종 데이터
merged

# excel 파일로 추출
# merged.to_excel("dataset/최종일사량합친데이터.xlsx", index=False, engine="openpyxl")